In [7]:
# pip install git+https://github.com/rcmalli/keras-vggface.git

  Cloning https://github.com/rcmalli/keras-vggface.git to /private/var/folders/jg/9shvmq_x3vx53ksfc_fg462w0000gn/T/pip-req-build-mj3qby9h
  Running command git clone --filter=blob:none --quiet https://github.com/rcmalli/keras-vggface.git /private/var/folders/jg/9shvmq_x3vx53ksfc_fg462w0000gn/T/pip-req-build-mj3qby9h
  Resolved https://github.com/rcmalli/keras-vggface.git to commit bee35376e76e35d00aeec503f2f242611a97b38a
  Preparing metadata (setup.py) ... done
Note: you may need to restart the kernel to use updated packages.


In [1]:
conda install Keras=2.2.4 tensorflow=1.14

Solving environment: - 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/osx-64::keras==2.2.4=0
failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/osx-64::keras==2.2.4=0
done


==> WARNING: A newer version of conda exists. <==
  current version: 4.10.3
  latest version: 23.7.4

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /opt/anaconda3/envs/VGG_py

  added / updated specs:
    - keras=2.2.4
    - tensorflow=1.14


The following NEW packages will be INSTALLED:

  keras-base         pkgs/main/osx-64::keras-base-2.2.4-py37_0


Preparing tr

In [2]:
pip install keras_vggface

Note: you may need to restart the kernel to use updated packages.


In [2]:
conda list tensorflow

# packages in environment at /opt/anaconda3/envs/VGG_py:
#
# Name                    Version                   Build  Channel
tensorflow                1.14.0          mkl_py37h085be34_0  
tensorflow-base           1.14.0          mkl_py37h5a24fda_0  
tensorflow-estimator      1.14.0                     py_0  

Note: you may need to restart the kernel to use updated packages.


In [3]:
conda list Keras

# packages in environment at /opt/anaconda3/envs/VGG_py:
#
# Name                    Version                   Build  Channel
keras                     2.2.4                         0  
keras-applications        1.0.8                      py_1  
keras-base                2.2.4                    py37_0  
keras-preprocessing       1.1.2              pyhd3eb1b0_0  
keras-vggface             0.6                      pypi_0    pypi

Note: you may need to restart the kernel to use updated packages.


In [4]:
# pip install --upgrade keras_vggface

In [5]:
from keras_vggface.vggface import VGGFace

In [6]:
import keras_vggface
print(keras_vggface.__version__)

0.6


In [3]:
pip install mtcnn

  Using cached mtcnn-0.1.1-py3-none-any.whl (2.3 MB)
  Obtaining dependency information for opencv-python>=4.1.0 from https://files.pythonhosted.org/packages/05/58/7ee92b21cb98689cbe28c69e3cf8ee51f261bfb6bc904ae578736d22d2e7/opencv_python-4.8.1.78-cp37-abi3-macosx_10_16_x86_64.whl.metadata
  Using cached opencv_python-4.8.1.78-cp37-abi3-macosx_10_16_x86_64.whl.metadata (19 kB)
Using cached opencv_python-4.8.1.78-cp37-abi3-macosx_10_16_x86_64.whl (54.7 MB)
Note: you may need to restart the kernel to use updated packages.


In [7]:
import mtcnn
print(mtcnn.__version__)

0.1.0


importing the dataset and cleaning function

In [8]:
import pandas as pd
import os

def filter_valid_relationships(relationships_df, base_path):
    valid_rows = []
    
    for index, row in relationships_df.iterrows():
        family_id = row['p1'].split("/")[0]
        person1_id = row['p1'].split("/")[1]
        person2_id = row['p2'].split("/")[1]

        person1_path = os.path.join(base_path, "train", family_id, person1_id)
        person2_path = os.path.join(base_path, "train", family_id, person2_id)
        
        if os.path.exists(person1_path) and os.path.exists(person2_path):
            valid_rows.append(row)
    
    return pd.DataFrame(valid_rows)

base_path = "/Users/fanyupei/Codes/CE4041_Kaggle/recognizing-faces-in-the-wild/"
relationships = pd.read_csv(base_path + "train_relationships.csv")
relationships = filter_valid_relationships(relationships, "/Users/fanyupei/Codes/CE4041_Kaggle/recognizing-faces-in-the-wild")

In [11]:
import numpy as np
from keras.preprocessing import image
from keras_vggface import utils
    
def read_img(path):
    try:
        img = image.load_img(path, target_size=(224, 224))
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img = utils.preprocess_input(img, version=1)  # or version=2 based on the model
        return img
    except Exception as e:
        print(f"Error reading {path}. Error: {e}")
        return None

def generate_data(relationships, base_path):
    X = []
    Y = []
    for i in range(len(relationships)):
        family_id = relationships.iloc[i, 0].split("/")[0]
        person1_id = relationships.iloc[i, 0].split("/")[1]
        person2_id = relationships.iloc[i, 1].split("/")[1]
        
        person1_images = os.listdir(os.path.join(base_path, "train", family_id, person1_id))
        person2_images = os.listdir(os.path.join(base_path, "train", family_id, person2_id))
        
        if not person1_images or not person2_images:
            # Skip if no images found for either person
            continue
        
        for img1 in person1_images:
            for img2 in person2_images:
                img1_path = os.path.join(base_path, "train", family_id, person1_id, img1)
                img2_path = os.path.join(base_path, "train", family_id, person2_id, img2)
                
                if os.path.exists(img1_path) and os.path.exists(img2_path):
                    img1_processed = read_img(img1_path)
                    img2_processed = read_img(img2_path)
                    
                    if img1_processed is not None and img2_processed is not None:
                        X.append([img1_processed, img2_processed])
                        Y.append(1)  # As per your provided data, all pairs in the CSV are related.
    
    X = np.array(X)
    Y = np.array(Y)
    return X, Y

X, Y = generate_data(relationships, base_path)

Error reading /Users/fanyupei/Codes/CE4041_Kaggle/recognizing-faces-in-the-wild/train/F0039/MID3/.DS_Store. Error: cannot identify image file <_io.BytesIO object at 0x7f88b0838170>
Error reading /Users/fanyupei/Codes/CE4041_Kaggle/recognizing-faces-in-the-wild/train/F0039/MID3/.DS_Store. Error: cannot identify image file <_io.BytesIO object at 0x7f88e171eb90>
Error reading /Users/fanyupei/Codes/CE4041_Kaggle/recognizing-faces-in-the-wild/train/F0039/MID3/.DS_Store. Error: cannot identify image file <_io.BytesIO object at 0x7f88b0838170>
Error reading /Users/fanyupei/Codes/CE4041_Kaggle/recognizing-faces-in-the-wild/train/F0039/MID3/.DS_Store. Error: cannot identify image file <_io.BytesIO object at 0x7f88b0838170>


: 

train the vgg face model

In [ ]:
from keras_vggface.vggface import VGGFace
from keras.layers import Flatten, Dense, Input, concatenate
from keras.models import Model

# Base model with VGG16 architecture
base_model = VGGFace(include_top=False)

for layer in base_model.layers[:-3]:
    layer.trainable = False

img1_input = Input(shape=(224, 224, 3))
img2_input = Input(shape=(224, 224, 3))

img1_features = base_model(img1_input)
img2_features = base_model(img2_input)

merged_features = concatenate([img1_features, img2_features], axis=-1)
merged_features = Flatten()(merged_features)
merged_features = Dense(512, activation='relu')(merged_features)
output = Dense(1, activation='sigmoid')(merged_features)

model = Model([img1_input, img2_input], output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit([X[:, 0], X[:, 1]], Y, epochs=10, batch_size=32, validation_split=0.2)

test the model

In [ ]:
def generate_test_data(base_path, submission_df):
    X = []
    pairs = []
    for _, row in submission_df.iterrows():
        img1_path = os.path.join(base_path, "test", row['img_pair'].split('-')[0])
        img2_path = os.path.join(base_path, "test", row['img_pair'].split('-')[1])
        
        img1_processed = read_img(img1_path)
        img2_processed = read_img(img2_path)

        if img1_processed is not None and img2_processed is not None:
            X.append([img1_processed, img2_processed])
            pairs.append(row['img_pair'])

    X = np.array(X)
    return X, pairs

submission_df = pd.read_csv(base_path + "sample_submission.csv")
X_test, test_pairs = generate_test_data(base_path, submission_df)

prediction

In [ ]:
# Predicting the test set
predictions = model.predict([X_test[:, 0], X_test[:, 1]])

# Preparing submission dataframe
submission_df = pd.DataFrame({'img_pair': test_pairs, 'is_related': predictions.ravel()})

# Creating a CSV for submission
submission_df.to_csv(base_path + "my_submission.csv", index=False)